# Phase 2 — Ablation Studies

**CSP-Ablation-Project** · placeholder

This notebook will contain targeted ablation experiments on the ethical circuits
identified in Phase 1.  

Planned experiments:
- Zero-out ablation of critical layers
- Activation patching between clean/corrupted examples
- Component-level analysis (attention heads, MLP sub-layers)

---

*To be implemented after Phase 1 results are validated.*

In [ ]:
# TODO: implement ablation pipeline